First, let's import the Python bindings, as usual.

In [7]:
import metapy

In [8]:
metapy.__version__ # you will want your version to be >= to this

'0.2.13'

If you would like to, you can inform MeTA to output log data to stderr like so:

In [9]:
metapy.log_to_stderr()

Now, let's download a list of stopwords and a sample dataset to begin exploring MeTA's topic models.

In [10]:
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

--2019-05-17 17:32:12--  https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.76.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.76.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747 (2.7K) [text/plain]
Saving to: ‘lemur-stopwords.txt’

lemur-stopwords.txt 100%[===================>]   2.68K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2019-05-17 17:32:14 (12.3 MB/s) - ‘lemur-stopwords.txt’ saved [2747/2747]



In [11]:
!wget -N https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
!tar xf ceeaus.tar.gz

--2019-05-17 17:32:16--  https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
Resolving meta-toolkit.org (meta-toolkit.org)... 50.116.41.177
Connecting to meta-toolkit.org (meta-toolkit.org)|50.116.41.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1182344 (1.1M) [application/octet-stream]
Saving to: ‘ceeaus.tar.gz’

ceeaus.tar.gz       100%[===================>]   1.13M   121KB/s    in 9.7s    

2019-05-17 17:32:28 (120 KB/s) - ‘ceeaus.tar.gz’ saved [1182344/1182344]



We will need to index our data to proceed. We eventually want to be able to extract the bag-of-words representation for our individual documents, so we will want a `ForwardIndex` in this case.

In [30]:
fidx = metapy.index.make_forward_index('review.toml')

 > Counting lines in file: [=================================] 100% ETA 00:00:00 
 
1558086132: [info]     Creating forward index: reviews-idx/fwd (/private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-nz_6bi23/deps/meta/src/index/forward_index.cpp:239)
1558086132: [info]     Creating forward index: reviews-idx/fwd (/private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-nz_6bi23/deps/meta/src/index/forward_index.cpp:239)
 > Tokenizing Docs: [>                                       ]   0% ETA 00:00:00  
 
1558086132: [warning]  Empty document (id = 5769) generated! (/private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-nz_6bi23/deps/meta/src/index/forward_index.cpp:335)
1558086132: [warning]  Empty document (id = 5769) generated! (/private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-nz_6bi23/deps/meta/src/index/forward_index.cpp:335)
 > Tokenizing Docs: [================>                       ]  41% ETA 00:00:02    
 


Just like in classification, the feature set used for the topic modeling will be the feature set used at the time of indexing, so if you want to play with a different set of features (like bigram words), you will need to re-index your data.

For now, we've just stuck with the default filter chain for unigram words, so we're operating in the traditional bag-of-words space.

Let's load our documents into memory to run the topic model inference now.

In [31]:
dset = metapy.learn.Dataset(fidx)

 > Loading instances into memory: [==========================] 100% ETA 00:00:00 
 


Now, let's try to find some topics for this dataset. To do so, we're going to use a generative model called a topic model.

There are many different topic models in the literature, but the most commonly used topic model is Latent Dirichlet Allocation. Here, we propose that there are K topics (represented with a categorical distribution over words) $\phi_k$ from which all of our documents are genereated. These K topics are modeled as being sampled from a Dirichlet distribution with parameter $\vec{\alpha}$. Then, to generate a document $d$, we first sample a distribution over the K topics $\theta_d$ from another Dirichlet distribution with parameter $\vec{\beta}$. Then, for each word in this document, we first sample a topic identifier $z \sim \theta_d$ and then the word by drawing from the topic we selected ($w \sim \phi_z$). Refer to the [Wikipedia article on LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) for more information.

The goal of running inference for an LDA model is to infer the latent variables $\phi_k$ and $\theta_d$ for all of the $K$ topics and $D$ documents, respectively. MeTA provides a number of different inference algorithms for LDA, as each one entails a different set of trade-offs (inference in LDA is intractable, so all inference algorithms are approximations; different algorithms entail different approximation guarantees, running times, and required memroy consumption). For now, let's run a Variational Infernce algorithm called CVB0 to find two topics. (In practice you will likely be finding many more topics than just two, but this is a very small toy dataset.)

In [ ]:
lda_inf = metapy.topics.LDACollapsedVB(dset, num_topics=10, alpha=1.0, beta=0.01)
lda_inf.run(num_iters=1000)

Initialization: [============================================] 100% ETA 00:00:00                        
 
Iteration 1 maximum change in gamma: 1.42598                                                                    
Iteration 1 maximum change in gamma: 1.42598                                    
Iteration 2 maximum change in gamma: 0.290918                                                                   
Iteration 2 maximum change in gamma: 0.290918                                   
Iteration 3 maximum change in gamma: 0.630436                                                                    
Iteration 3 maximum change in gamma: 0.630436                                   
Iteration 4 maximum change in gamma: 0.981627                                                                   
Iteration 4 maximum change in gamma: 0.981627                                    
Iteration 5 maximum change in gamma: 1.10921                                                                    
It

The above ran the CVB0 algorithm for 1000 iterations, or until an algorithm-specific convergence criterion was met. Now let's save the current estimate for our topics and topic proportions.

In [16]:
lda_inf.save('lda-cvb0')

We can interrogate the topic inference results by using the `TopicModel` query class. Let's load our inference results back in.

In [17]:
model = metapy.topics.TopicModel('lda-cvb0')

 > Loading topic term probabilities: [=======================] 100% ETA 00:00:00 
 
 > Loading document topic probabilities: [===================] 100% ETA 00:00:00 
 


Now, let's have a look at our topics. A typical way of doing this is to print the top $k$ words in each topic, so let's do that.

In [18]:
model.top_k(tid=0)

[(3759, 0.06705638847185073),
 (1968, 0.05605922553333449),
 (2635, 0.05222299369860683),
 (3549, 0.046429323016541464),
 (665, 0.03488136097176335),
 (4157, 0.02906744262754168),
 (2322, 0.028850181393749164),
 (3729, 0.022331330009265446),
 (1790, 0.020755677124097673),
 (3554, 0.015483015030917832)]

The models operate on term ids instead of raw text strings, so let's convert this to a human readable format by using the vocabulary contained in our `ForwardIndex` to map the term ids to strings.

In [19]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0)]

[('time', 0.06705638847185073),
 ('job', 0.05605922553333449),
 ('part', 0.05222299369860683),
 ('student', 0.046429323016541464),
 ('colleg', 0.03488136097176335),
 ('work', 0.02906744262754168),
 ('money', 0.028850181393749164),
 ('think', 0.022331330009265446),
 ('import', 0.020755677124097673),
 ('studi', 0.015483015030917832)]

In [20]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1)]

[('smoke', 0.13110415371161654),
 ('restaur', 0.054349396306202634),
 ('peopl', 0.03678012902347797),
 ('smoker', 0.033492686732128525),
 ('ban', 0.022530705207015563),
 ('think', 0.015620494401503484),
 ('japan', 0.012780936422655962),
 ('complet', 0.012635087335998278),
 ('cigarett', 0.01198720005093396),
 ('non', 0.011317756210677255)]

We can pretty clearly see that this particular dataset was about two major issues: smoking in public and part time jobs for students. This dataset is actually a collection of essays written by students, and there just so happen to be two different topics they can choose from!

The topics are pretty clear in this case, but in some cases it is also useful to score the terms in a topic using some function of the probability of the word in the topic and the probability of the word in the other topics. Intuitively, we might want to select words from each topic that best reflect that topic's content by picking words that both have high probability in that topic **and** have low probability in the other topics. In other words, we want to balance between high probability terms and highly specific terms (this is kind of like a tf-idf weighting). One such scoring function is provided by the toolkit in `BLTermScorer`, which implements a scoring function proposed by Blei and Lafferty.

In [21]:
scorer = metapy.topics.BLTermScorer(model)
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0, scorer=scorer)]

[('job', 0.34821994022160396),
 ('part', 0.31311031896230457),
 ('student', 0.2832889530312013),
 ('colleg', 0.20808962299809064),
 ('time', 0.17797907086590925),
 ('money', 0.16234660557757263),
 ('work', 0.15585417209548247),
 ('studi', 0.08228282097843165),
 ('learn', 0.06491891126184335),
 ('experi', 0.05494481108549043)]

In [22]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1, scorer=scorer)]

[('smoke', 0.8741657378126444),
 ('restaur', 0.31746084754386894),
 ('smoker', 0.20060312729691665),
 ('ban', 0.12853062376878555),
 ('cigarett', 0.0655761575981626),
 ('non', 0.061284307367810875),
 ('complet', 0.061053836988573806),
 ('japan', 0.05846410633716781),
 ('health', 0.050548412383300656),
 ('seat', 0.0453399574637634)]

Here we can see that the uninformative word stem "think" was downweighted from the word list from each topic, since it had relatively high probability in either topic.

We can also see the inferred topic distribution for each document.

In [23]:
model.topic_distribution(0)

<metapy.stats.Multinomial {0: 0.978659, 1: 0.021341}>

It looks like our first document was written by a student who chose the part-time job essay topic...

In [24]:
model.topic_distribution(900)

<metapy.stats.Multinomial {0: 0.021203, 1: 0.978797}>

...whereas this document looks like it was written by a student who chose the public smoking essay topic.

We can also infer topics for a brand new document. First, let's create the document and use the forward index we loaded before to convert it to a feature vector:

In [25]:
doc = metapy.index.Document()
doc.content("I think smoking in public is bad for others' health.")
fvec = fidx.tokenize(doc)

Now, let's load a topic model inferencer that uses the same CVB inference method we used earlier:

In [26]:
inferencer = metapy.topics.CVBInferencer('lda-cvb0.phi.bin', alpha=1.0)

 > Loading topic term probabilities: [=======================] 100% ETA 00:00:00 
 


Now, let's infer the topic proportions for the new document:

In [27]:
proportions = inferencer.infer(fvec, max_iters=20, convergence=1e-4)
print(proportions)

<metapy.stats.Multinomial {0: 0.185608, 1: 0.814392}>
